In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from torchvision import transforms
from piq import ssim
import torch
import torch.nn as nn
from PIL import Image


In [ ]:
# 디스플레이 및 저장 함수

def display_image(image):
    if image is not None:
        if len(image.shape) == 3 and image.shape[2] == 3:
            # print('3채널 이미지를 표시합니다.')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            plt.imshow(image)
            
        else:
            # 이미지가 1채널인 경우 그대로 사용
            # print('1채널 이미지를 표시합니다')
            plt.imshow(image, cmap='gray')
    else:
        # 이미지가 없을 때
        print('이미지가 존재하지 않습니다.')



def save_image(image, filename='new_image'):
    if image is not None:
        # 이미지가 BGR 형식이면 그레이스케일로 변환
        if len(image.shape) == 3 and image.shape[2] == 3:
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            # 이미지가 이미 그레이스케일인 경우
            gray_image = image

        # 이미지를 저장
        cv2.imwrite(f"./output/{filename}.png", gray_image)
        print(f'Saved...{filename}')
# 에지 강조 함수
def emphasize_edge(image, low=50, upper=150):
    # 이미지 로드 및 에지 검출, 이진화
    original_image = image
    gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, low, upper)

    # 에지의 윤곽을 찾음
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # 에지 안쪽을 색칠하기 위한 빈 이미지 생성
    filled_image = np.zeros_like(original_image)
    # 에지 윤곽을 그림
    cv2.drawContours(filled_image, contours, -1, (255, 255, 255), thickness=cv2.FILLED)
    # 색칠된 이미지와 원본 이미지를 합침
    result_image = cv2.addWeighted(original_image, 1, filled_image, 0.5, 0)
    # 결과 이미지를 저장하거나 표시
    # print('에지 강조 결과를 표시합니다.')
    display_image(result_image)
    
    return result_image


# 이미지 흑백화 + 이진화
def convert_to_binary(img, threshold):
    
    # 이미지 이진화
    _, binary_img = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    # print('이진화 결과를 표시합니다.')
    display_image(binary_img)

    return binary_img
    


# 밝기 조정 함수 (조절 정도 -100부터 100까지)
def adjust_brightness(img, brightness_factor=30):
    # 밝기 조절
    adjusted_image = cv2.convertScaleAbs(img, alpha=1, beta=brightness_factor)
    # print('밝기 조정 결과를 표시합니다.')
    display_image(adjusted_image)

    return adjusted_image
    
    

# 대비 조정 함수 (-100부터 100까지)
def adjust_contrast(img, contrast_factor=1.5):

    # 대비 조절
    adjusted_image = cv2.convertScaleAbs(img, alpha=contrast_factor)
    # print('대비 조정 결과를 표시합니다.')
    display_image(adjusted_image)

    return adjusted_image

# 색 반전 함수
def invert_colors(image, count):
    count += 1
    
    inverted = 255 - image
    new_image = inverted + count *image // (count + 1)
    
    # print('색 반전 결과를 표시합니다.')
    display_image(new_image)
    
    return count, new_image


# 작은 노이즈 제거 (모폴로지)
def remove_small_noise(binary_image, diameter_threshold_mm=0.8):
    
    # 모폴로지 연산을 위한 구조 요소 생성 (원형)
    kernel_size = int((diameter_threshold_mm / 2) * 10)  # 직경이 1mm 미만인 경우를 고려하여 크기 조정
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    # Opening 연산을 통해 작은 노이즈 제거
    result_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel)
    
    # print('모폴로지 연산 결과를 표시합니다.')
    display_image(result_image)
    
    return result_image


def grayscale_image(image):
    
    # 현재 이미지가 그레이스케일인지 확인
    is_gray = len(image.shape) == 2 or (len(image.shape) == 3 and image.shape[2] == 1)
    
    if is_gray:
        # 이미지가 그레이스케일이면 RGB로 변환
        image = image
    else:
        # 이미지가 RGB이면 그레이스케일로 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # print('그레이스케일 변환 결과를 표시합니다.')
    display_image(image)
    
    return image
# psnr, ssim 점수

def psnr(input, target, data_range=1.0):
    mse = torch.mean((input - target) ** 2)
    if mse == 0:
        return float('inf')
    return 10 * torch.log10((data_range ** 2) / mse)


def psnr_ssim(image, target_file):
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    target_image = Image.open(target_file).convert("RGB")
    
    preprocess = transforms.Compose([transforms.ToTensor()])
    transform = transforms.Compose([transforms.Resize((image.shape[0], image.shape[1]))])
    
    image = preprocess(image)
    target_image = preprocess(transform(target_image))
    image = image.unsqueeze(0)  # 차원 추가
    target_image = target_image.unsqueeze(0)  # 차원 추가
    
    psnr_value = psnr(image, target_image, data_range=1.0)
    ssim_value = ssim(image, target_image, data_range=1.0)
    # print(f'psnr: {psnr_value}, ssim: {ssim_value}')

    return psnr_value, ssim_value



In [ ]:
input_folder = '/home/piai/문서/miryeong/Algorithm_1/input'
target_folder = '/home/piai/문서/miryeong/Algorithm_1/target'

In [ ]:
# 함수 종류
# emphasize_edge(image, low=50, upper=150)
# convert_to_binary(img, threshold)
# adjust_brightness(img, brightness_factor=30)
# adjust_contrast(img, contrast_factor=1.5)
# invert_colors(image, count)
# remove_small_noise(binary_image, diameter_threshold_mm=0.8)
# grayscale_image(image)
    
    
invert_counts = 0


for filename in os.listdir(input_folder):
    print(f'Processing...{filename}')
    
    image_path = os.path.join(input_folder, filename)
    image = cv2.imread(image_path)
    
    # 전처리 함수 호출
    invert_counts, image = invert_colors(image, invert_counts)
    image = adjust_brightness(image, brightness_factor=-10)
    image = adjust_contrast(image, contrast_factor=1)
    image = grayscale_image(image)
    image = convert_to_binary(image, threshold=200)
    image = remove_small_noise(image, diameter_threshold_mm=0.6)
    
    target_file = os.path.join(target_folder, os.path.basename(filename))
    
    if os.path.exists(target_file):
        # 평가 지표 계산
        v_psnr, v_ssim = psnr_ssim(image, target_file)
        
    else:
        print('target 이미지가 존재하지 않습니다.')
        
    
    # 결과물 저장
    save_image(image, filename=filename)

In [ ]:
# 테스트 이미지 생성
input_image = cv2.imread('/home/piai/문서/miryeong/Algorithm_1/input/saved_image1.png')

# PSNR을 계산하고 최적의 분기 조건 찾기
best_psnr = -100
best_params = {}


# 이미지를 개선하는 함수
def enhance_image(input_image, brightness, threshold, contrast):

    # 각각의 파라미터에 따라 이미지를 개선
    enhanced_image = adjust_brightness(input_image, brightness_factor=brightness)
    enhanced_image = adjust_contrast(enhanced_image, contrast_factor=contrast)
    enhanced_image = grayscale_image(enhanced_image)
    enhanced_image = convert_to_binary(enhanced_image, threshold=threshold)

    return enhanced_image                                                                                                                                                                                                                                                                                                                                                                                               



for brightness in range(0, 101, 20):
    for threshold in range(0, 256, 50):
        for contrast in np.arange(0, 2.1, 1.0):
            
            # print(f'Processing : brightness:{brightness}, threshold:{threshold}, contrast:{contrast}')
            # enhanced_image = enhance_image(input_image, brightness, threshold, contrast)
            # psnr_value, ssim_value = psnr_ssim(enhanced_image, '/home/piai/문서/miryeong/Algorithm_1/target/saved_image1.png')
            # current_psnr = psnr_value
            
            try:
                enhanced_image = enhance_image(input_image, brightness, threshold, contrast)
                psnr_value, ssim_value = psnr_ssim(enhanced_image, '/home/piai/문서/miryeong/Algorithm_1/target/saved_image1.png')
                current_psnr = psnr_value
                
                if current_psnr > best_psnr:
                    best_psnr = current_psnr
                    best_params = {'brightness': brightness, 'threshold': threshold, 'contrast': contrast}
                    
                    print(f'*********best psnr is {best_psnr:.3f} with {best_params}')
                    save_image(image= enhanced_image)
            
            except Exception as e:
                print(f'Exception occurred: {e}')
                
                
print(f'Final result: best_psnr={best_psnr}, best_params={best_params}')
